In [50]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib as mpl


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.copy_on_write = True

In [51]:
con = sqlite3.Connection('./radio_ads.db')
cur = con.cursor()

In [52]:
rok_s = 2022
rok_e = 2023
mc_s = 1
mc_e = 12
dzi_s = 1
dzi_e = 31

In [53]:
query = f'''SELECT "data_czas"."data" AS "data", "dzien", "dzien_tyg_nr" AS 'd_tyg_nr', "dzien_tyg", 
"tydzien", "miesiac_nr" AS "m_nr", "miesiac", "rok", "spoty"."kod_reklamy" AS "kod_reklamy", "brand", 
"submedium", "nadawca", "zasieg", "godz_bloku_rek" AS "godz_bloku", "daypart", 
"dl_ujednolicona" AS "dl_spotu", "typ_produktu", "koszt", "typ", "l_emisji" AS "ilosc"
FROM "spoty"
JOIN "data_czas" ON "data_czas"."data" = "spoty"."data"
JOIN "brandy" ON "brandy"."id" = "spoty"."brand_id"
JOIN "submedia" ON "submedia"."id" = "spoty"."submedium_id"
JOIN "nadawcy" ON "nadawcy"."id" = "submedia"."nadawca_id"
JOIN "zasiegi" ON "zasiegi"."id" = "submedia"."zasieg_id"
JOIN "czasy_reklam" ON "czasy_reklam"."id" = "spoty"."czas_reklamy_id"
JOIN "dayparty" ON "dayparty"."id" = "czasy_reklam"."daypart_id"
JOIN "dl_ujednolicone" ON "dl_ujednolicone"."id" = "czasy_reklam"."dl_ujednolicona_id"
JOIN "typy_produktu" ON "typy_produktu"."id" = "spoty"."typ_produktu_id"
JOIN "dni_tyg" ON "dni_tyg"."id" = "data_czas"."dzien_tyg_nr"
JOIN "miesiace" ON "miesiace"."id" = "data_czas"."miesiac_nr"
WHERE "miesiac_nr" BETWEEN {mc_s} AND {mc_e} AND "rok" BETWEEN {rok_s} AND {rok_e} AND "dzien" BETWEEN {dzi_s} AND {dzi_e};'''

In [54]:
df = pd.read_sql(sql=query, con=con)
df.head(1)

,data,dzien,d_tyg_nr,dzien_tyg,tydzien,m_nr,miesiac,rok,kod_reklamy,brand,submedium,nadawca,zasieg,godz_bloku,daypart,dl_spotu,typ_produktu,koszt,typ,ilosc
0,2022-01-01,1,6,Sobota,0,1,Styczeń,2022,22126230,EURO RTV AGD,ESKA Koszalin,TIME,miejskie,17:00-17:29,po 16,30,GRUPA,100,reklama,1


In [56]:
dziennie = df[['brand', 'submedium', 'ilosc', 'dzien', 'dzien_tyg', 'data', 'rok', 'miesiac']]
dziennie.loc[:, 'dzien_tyg'] = dziennie['dzien_tyg'].str[:2]
dziennie['medium'] = 'radio'
dziennie.head()

,brand,submedium,ilosc,dzien,dzien_tyg,data,rok,miesiac,medium
0,EURO RTV AGD,ESKA Koszalin,1,1,So,2022-01-01,2022,Styczeń,radio
1,EURO RTV AGD,VOX FM,1,1,So,2022-01-01,2022,Styczeń,radio
2,EURO RTV AGD,ESKA Koszalin,1,1,So,2022-01-01,2022,Styczeń,radio
3,EURO RTV AGD,ESKA Koszalin,1,1,So,2022-01-01,2022,Styczeń,radio
4,EURO RTV AGD,VOX FM,1,1,So,2022-01-01,2022,Styczeń,radio


In [57]:
stacje = ['ESKA Wrocław', 'ZET', 'RMF FM', 'TOK FM', 'PR 1']
rok = (2023, 2022)
miesiac = 'Styczeń'
dzien = 15
filtr = (dziennie['rok'].isin(rok)) & (dziennie['miesiac'] == miesiac) & (dziennie['submedium'].isin(stacje)) & (dziennie['dzien'] <= dzien) 
dziennie = dziennie.loc[filtr]
dziennie.set_index(['submedium', 'brand'], inplace=True)
dziennie = dziennie.sort_index(ascending=True)
tp = dziennie.pivot_table(values='ilosc', index=['submedium', 'brand'], columns=['rok', 'miesiac','dzien', 'dzien_tyg'],aggfunc='sum', fill_value=0)
tp

rok                            2022                                          \
miesiac                     Styczeń                                           
dzien                            1   2   3   4   5   6   7   8   9   10  11   
dzien_tyg                        So  Ni  Po  Wt  Śr  Cz  Pi  So  Ni  Po  Wt   
submedium    brand                                                            
ESKA Wrocław EURO RTV AGD        20  28  32  30  30  32  32  32  32  28  37   
             MEDIA EXPERT         8  10  16  16  16  11  17  11   9  16  16   
             MEDIA MARKT          0   3  12   6  10   3  12   7   0   9   0   
             NEONET AGD RTV      12  12  18  14  14  18  18  16  12  18  14   
PR 1         EURO RTV AGD        18  22  22  18  18  18  22  22  22  22  22   
             MEDIA EXPERT        15  24  32  32  31  20  31  30  25  32  32   
             NEONET AGD RTV      25  20  24  24  24  24  24  25  20  24  24   
RMF FM       EURO RTV AGD        28  32  40  33  34  38  38  38  38  32  45   
             MEDIA EXPERT        20  28  31  32  32  19  32  32  32  32  32   
             MEDIA MARKT          0   4  10   3  10   3  10   3   3   8   2   
TOK FM       EURO RTV AGD        18  22  26  20  20  20  26  26  26  22  22   
             MEDIA EXPERT        12  19  25  23  23  15  21  20  23  24  24   
             MEDIA MARKT          0   0  16   7   9   0  16   6   0   0   0   
ZET          EURO RTV AGD        20  28  32  30  30  32  32  32  32  28  39   
             MEDIA EXPERT        21  31  36  36  36  21  35  32  31  36  36   
             MEDIA MARKT          0   4  10   3   8   2   8   5   0   7   3   
             NEONET AGD RTV      13  15  31  24  24  30  30  17  15  31  24   

rok                                            2023                          \
miesiac                                     Styczeń                           
dzien                        12  13  14  15      1   2   3   4   5   6   7    
dzien_tyg                    Śr  Cz  Pi  So      Ni  Po  Wt  Śr  Cz  Pi  So   
submedium    brand                                                            
ESKA Wrocław EURO RTV AGD    39  28  39  41       0  30  30  30  30  23  28   
             MEDIA EXPERT    16  17  17  11       8  18  19  19  19  16  12   
             MEDIA MARKT      0   7   7   5       0   7   7   3  10   2   2   
             NEONET AGD RTV  14  18  18  16      15  15  15  15  15  15  15   
PR 1         EURO RTV AGD    18  17  19  19       0   0   0   0   0   0   0   
             MEDIA EXPERT    32  32  32  30      13  31  30  30  30  24  28   
             NEONET AGD RTV  24  24  24  25      20  21  20  20  20  20  20   
RMF FM       EURO RTV AGD    45  34  46  46       0  43  43  43  43  36  42   
             MEDIA EXPERT    32  32  32  32      13  31  31  31  31  27  32   
             MEDIA MARKT      0   6  12   5       0   4   4   0   6   2   3   
TOK FM       EURO RTV AGD    20  17  19  19       0  19  19   0  19  15  20   
             MEDIA EXPERT    24  24  22  23      16  23  22  23  23  20  13   
             MEDIA MARKT      0   6   8   6       0  12  12   6   8   2   6   
ZET          EURO RTV AGD    37  30  38  42       0  35  35  33  35  28  34   
             MEDIA EXPERT    36  35  32  28      23  32  32  32  32  32  29   
             MEDIA MARKT      0   0  14   1       0   4   0   2   2   2   0   
             NEONET AGD RTV  24  30  30  17      34  33  33  33  33  33  35   

rok                                                          
miesiac                                                      
dzien                        8   9   10  11  12  13  14  15  
dzien_tyg                    Ni  Po  Wt  Śr  Cz  Pi  So  Ni  
submedium    brand                                           
ESKA Wrocław EURO RTV AGD    25  35  35  30  15  20  20  20  
             MEDIA EXPERT     9  18  19  19  20  20  11   7  
             MEDIA MARKT      2   2   0   8   8   9   3   2  
             NEONET AGD RT

In [58]:
tp.diff(axis=1, periods=15).style.format("{:.0f}", na_rep="-")

In [59]:
def highlight_max(s):
    '''
    highlight the maximum in a Series green.
    '''
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

def highlight_min(s):
    '''
    highlight the maximum in a Series red.
    '''
    is_min = s == s.min()
    return ['background-color: brown' if v else '' for v in is_min]

tp.pct_change(axis='columns', periods=15).style.format('{:.0%}', na_rep='-').apply(highlight_min)

In [94]:
dziennie = df[['brand', 'submedium', 'ilosc', 'dzien', 'dzien_tyg', 'data', 'rok', 'miesiac']]
dziennie.loc[:, 'dzien_tyg'] = dziennie['dzien_tyg'].str[:2]
dziennie['medium'] = 'radio'
dziennie.head()

stacje = ['ESKA Wrocław', 'ZET', 'RMF FM', 'TOK FM', 'PR 1']
rok = (2022, 2023)
miesiac = ('Styczeń',)
dzien = 31


filtr = (dziennie['submedium'].isin(stacje)) & (dziennie['rok'].isin(rok)) & (dziennie['miesiac'].isin(miesiac))
dziennie = dziennie.loc[filtr]
dziennie.set_index(['brand', 'medium'], inplace=True)
dziennie = dziennie.sort_index(ascending=True)
tp = dziennie.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc=sum)
tp['roznica'] = tp[2023] - tp[2022]
tp['dynamika'] = tp[2023] / tp[2022] - 1
tp.style \
  .format(thousands=' ', decimal=',') \
  .format('{:.1%}', subset=['dynamika'])


/tmp/ipykernel_17112/387076115.py:16: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  tp = dziennie.pivot_table(values='ilosc', index=['brand', 'medium'], columns=['rok', 'miesiac'], aggfunc=sum)


,rok,2022,2023,roznica,dynamika
,miesiac,Styczeń,Styczeń,,
brand,medium,,,,
EURO RTV AGD,radio,4 243,2 881,-1 362,-32.1%
MEDIA EXPERT,radio,4 022,3 751,-271,-6.7%
MEDIA MARKT,radio,625,420,-205,-32.8%
NEONET AGD RTV,radio,2 110,2 098,-12,-0.6%
